In [9]:
import numpy as np
from collections import defaultdict as dd
from typing import List, Dict, Set
import nltk
from tqdm import tqdm_notebook, tnrange
import pickle
from termcolor import colored
import sqlite3

In [10]:
conn = sqlite3.connect("Dane/wikipedyjka.db")

In [11]:
def to_lemma_mapping():
    all_lemmas = dd(list)
    for line in open('Dane/polimorfologik-2.1.txt', encoding='utf-8'):
        L = line.split(';')[:2]
        all_lemmas[L[1].lower()].append(L[0].lower())
    return all_lemmas
lemma_mapping = to_lemma_mapping()

In [12]:
wiki_list = []
key = ""
with open("Dane/fp_wiki.txt") as f:
    cntr = -1
    for line in f:
        tokenized_line = line.split(": ")
        if tokenized_line[0] == "TITLE":
            if tokenized_line[-1][-1] == "\n":
                tokenized_line[-1] = tokenized_line[-1][:-1]
            cntr+=1
            wiki_list.append((tokenized_line[-1], []))
            continue
        wiki_list[-1][1].append(line)

In [14]:
cntr = 0
positional_index = dd(list)
articles_pos = []
for title, article in wiki_list:
    cntr += 1
    articles_pos.append(cntr)
    for line in article:
        tokenized_line = nltk.word_tokenize(line)
        for w in tokenized_line:
            for lemma in lemma_mapping[w]:
                positional_index[lemma].append(cntr)
            cntr += 1

KeyboardInterrupt: 

In [ ]:
c = conn.cursor()
for k, v in positional_index:
    c.execute(f'''CREATE TABLE {k}
             (position INTEGER)''')
    for ind in v:
        c.execute(f"INSERT INTO {k} VALUES ({ind})")
c.commit()